# Factory classes

Some myHDL libraries provide classes to generate `@block` components. These are currently under scrutiny. 



In [1]:
import sys
sys.path.insert(0, '../..')

## MyHDL factory classes (emulation)

For the time being, factory classes have be decorated using the `@factory` decorator in order to be translated to IRL.
Not that the following class makes sense in the real world, but as an example:

### `@block` factory

In [2]:
from myirl.emulation import *
from myirl.kernel.components import component, DesignModule

@factory
class Bingo:
    def __init__(self, parameter = 8):
        self.n = parameter
        
        T = Signal.Type(intbv, parameter)
    
        @block
        def my_unit(clk : ClkSignal, a : T, b : T.Output):
            
            @always(clk.posedge)
            def worker():
                b.next = a

            return instances()

        self.my_unit = my_unit        

In [3]:
def test():
    b = Bingo(6)
    u, v = [Signal(intbv()[6:]) for _ in range(2)]
    c = ClkSignal()
    u = b.my_unit(c, u, v)
    
    f = u.elab(targets.VHDL)
    
test()

Creating process 'my_unit/worker' with sensitivity (clk'rising,)
 Insert unit my_unit_s1_s6_s6 
 Writing 'my_unit' to file /tmp/my_unit.vhdl 
Finished _elab in 0.0020 secs


In [4]:
!cat /tmp/my_unit.vhdl

-- File generated from /usr/local/lib/python3.10/runpy.py
-- (c) 2016-2021 section5.ch
-- Modifications may be lost

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.txt_util.all;
use work.myirl_conversion.all;

entity my_unit is
    port (
        clk : in std_ulogic;
        a : in unsigned(5 downto 0);
        b : out unsigned(5 downto 0)
    );
end entity my_unit;

architecture MyIRL of my_unit is
    -- Local type declarations
    -- Signal declarations
begin
    
worker:
    process(clk)
    begin
        if rising_edge(clk) then
            b <= a;
        end if;
    end process;
end architecture MyIRL;



### Class with static `@block_component` member

When specified as a static unit inside this class, the class will serve as its context and the unit requires a `@block_component` decoration (without arguments), plus the `self` argument, representing a *component method* describing hardware inference.
In this case, the factory class is derived from the `DesignModule` (or minimum the `BareModule` class). This construct helps to encapsulate common blocks that depend on a complex configuration.

In [5]:
from myirl.kernel import components
from myirl.test import common_test

@factory
class Bongo(components.DesignModule):
    def __init__(self, parameter = 8, name = "Bongo"):
        self.n = parameter
        super().__init__(name)
        self.files = []
        self.inst = None
    
    # Here, we use `@block_component` without arguments, as the context is 'self'
    @block_component
    def my_unit1(self, clk : ClkSignal, a : Signal, b : Signal.Output):
        "A component somewhere in the hierarchy"
        ls = Signal(intbv()[self.n:])
        
        @always(clk.posedge)
        def worker():
            ls.next = a
            b.next = ls

        return instances()
    
    @block_component
    def my_tb(self):
        "A built-in test bench"
        u, v = [Signal(intbv()[self.n:]) for _ in range(2)]
        c = ClkSignal()
        
        @always(delay(4))
        def clkgen():
            c.next = ~c
            
        @instance
        def stim():
            u.next = 2
            yield c.posedge
            yield c.posedge
            yield c.posedge
            print(v)
            assert v == 2
            raise StopSimulation
        
        uut = self.my_unit1(c, u, v)
        return instances()
    
    def build(self):
        t = self.my_tb()
        self.files = t.elab(targets.VHDL, elab_all = True)
        self.inst = t
        print("Resulting files", self.files)
        
    def simulate(self):
        common_test.run_ghdl(self.files, self.inst, debug = True)
        
#print(Bongo.unparse())

Running this design component, *OO* style:

In [6]:
def test():
    b = Bongo(6)
    b.build()
    b.simulate()
    
    b = Bongo(7)
    b.build()
test()

Using default for name: Bongo
 Declare obj 'my_tb' in context 'Bongo' 
Creating process 'my_tb/clkgen' with sensitivity (<myirl.kernel.sensitivity.DeltaT object at 0x7f2ac4782fb0>,)
Creating sequential 'my_tb/stim' 
 Declare obj 'my_unit1' in context 'Bongo' 
Creating process 'my_unit1/worker' with sensitivity (clk'rising,)
 Writing 'my_unit1' to file /tmp/myirlg5rhqq4c/my_unit1.vhdl 
Finished _elab in 0.0008 secs
 Writing 'my_tb' to file /tmp/myirlg5rhqq4c/my_tb.vhdl 
Finished _elab in 0.0013 secs
 Creating library file /tmp/myirlq9ekr3cr/module_defs.vhdl 
Resulting files ['/tmp/myirlg5rhqq4c/my_unit1.vhdl', '/tmp/myirlg5rhqq4c/my_tb.vhdl', '/tmp/myirlq9ekr3cr/module_defs.vhdl']
==== COSIM stdout ====

==== COSIM stderr ====

==== COSIM stdout ====
analyze /home/testing/src/myhdl2/myhdl.v2we/examples/../../myirl/targets/../test/vhdl/txt_util.vhdl
analyze /home/testing/src/myhdl2/myhdl.v2we/examples/../../myirl/targets/libmyirl.vhdl
analyze /tmp/myirlg5rhqq4c/my_unit1.vhdl
analyze /tmp